# Иcточники данных

- Внешние источники
  - b1: `b1_analytics.pkl`
  - kamaflow: `kamaflow_researches.pkl`
- Внутренние источники
  - Отраслевой хаб: данные выложены в sberdisk

## Формат данных 

Результат представляет собой массив словарей, где каждый словарь содержит информацию об одном аналитическом материале. Каждый словарь имеет следующие ключи:

### Пример элемента списка `analytics`:
```json
{
    "link": "/news/12345",
    "date": "2023-10-15",
    "name": "Анализ рынка ИТ в 2023 году",
    "name_en": "it-market-analysis-2023",
    "tags": ["ИТ", "аналитика", "2023"],
    "pdf_links": [
        "https://b1.ru/local/assets/surveys/it-market-analysis-2023.pdf"
    ],
    "text": "В 2023 году рынок ИТ показал рост на 14%...",
    "pdfs": [
        {
            "name": "it-market-analysis-2023.pdf",
            "content": "binary_pdf_content_here"
        }
    ]
}
```
Мы хотим получить такой, при этом часть полей можент быть пустой:
```json
{
    "link": "/news/12345",
    "date": "2023-10-15",
    "name": "Анализ рынка ИТ в 2023 году",
    "name_en": "it-market-analysis-2023",
    "tags": ["ИТ", "аналитика", "2023"],
    "sber_tags": ["Теги", "из", "сбера", "отрасль", "из", "kksb_clinet_profile"],
    "pdf_links": [
        "https://b1.ru/local/assets/surveys/it-market-analysis-2023.pdf"
    ],
    "text": "В 2023 году рынок ИТ показал рост на 14%...",
    "summary": "Саммари на основе pdfs",
    "pdfs": [
        {
            "name": "it-market-analysis-2023.pdf",
            "content": "binary_pdf_content_here",
            "full_text": "Полный текст и pdf, то что получилось вытащить из content",
            "summary": "Саммари из full_text",
        }
    ]
}
```

Для дальнейнего анализа нужно только следующие поля: date, sber_tags, summary, pdfs['full_text'], pdfs['summary']:
```json
{
    "date": "2023-10-15",
    "sber_tags": ["Теги", "из", "сбера", "отрасль", "из", "kksb_clinet_profile"],
    "summary": "Саммари",
    "pdfs": [
        {
            "full_text": "Полный текст и pdf, то что получилось вытащить из content",
            "summary": "Саммари из full_text",
        },
        {
            "full_text": "Полный текст и pdf, то что получилось вытащить из content",
            "summary": "Саммари из full_text",
        }
    ]
}
```

In [1]:
import pickle


with open('b1_analytics.pkl', 'rb') as f:
    b1_analytics = pickle.load(f)

with open('kamaflow_researches.pkl', 'rb') as f:
    kamaflow_researches = pickle.load(f)

with open('industry_hub_analytics_update.pkl', 'rb') as f:
    industry_hub_analytics = pickle.load(f)


docs = b1_analytics + kamaflow_researches + industry_hub_analytics

## Добавляем саммари

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.chains.summarize import load_summarize_chain
from langchain.document_loaders import PyPDFLoader
from langchain.schema import Document
import tempfile
from tqdm import tqdm


llm = ChatOpenAI(model_name="gpt-4o-mini")

for doc in tqdm(docs):
    name = doc['name']
    tags = doc['tags']
    additional_text = doc['text']
    
    text_documents = []
    
    for pdf in doc['pdfs']:
        pdf_name = pdf['name']

        pdf_content = pdf.get('content', None)
        full_text = pdf.get('full_text', None)
        
        if full_text is None and pdf_content is not None:
            try:
                with tempfile.NamedTemporaryFile() as temp_file:
                    temp_file.write(pdf_content)
                    temp_file_path = temp_file.name
                    loader = PyPDFLoader(temp_file_path)
                    pdf_data = loader.load()
            except Exception as e:
                print(f"Error processing PDF {pdf_name}: {e}")
                pdf_data = None
            if pdf_data:
                full_text = "\n".join([doc.page_content for doc in pdf_data])
                pdf['full_text'] = full_text
        
        if full_text:
            if additional_text is None or len(additional_text) == 0:
                additional_text = 'нет краткого описание'
            text_document = f'**Документ с наименованием {pdf_name}, его содержимое:** {full_text}'
            text_documents.append(text_document)
            
            prompt = f"""
            **Название документа:**  {name}
            **Теги документа:** {", ".join(tags)} 
            **Краткое содержание документа с сайта (возможны html артефакты):** {additional_text}

            Перед тобой текст, полученный из PDF-документа. Он может содержать пропуски, так как изображения и таблицы не были извлечены. В этом тексте отраслевой анализ и отреслевыен отчёты.
            ```
                {text_document}
            ```


            **Твоя задача** - сделать саммари или краткое содержание этого отчёта, провести подробный и детальный отраслевой анализ на основе этого текста на русском языке. Не жалей букв, достань и саммаризируй всю полезную информацию. 
            Если информация обрывочна, попытайся восстановить логический контекст или отметь, какие данные отсутствуют.
            Данная информация потом будет использоваться для формирования стратегии компании.

            **Формат анализа:**
            1. **Ключевые выводы** - выдели главных мыслей.
            2. **Тенденции в отрасли** - связаны ли идеи с актуальными событиями или трендами? краткосрочные и долгосрочные прогнозы.
            3. **Анализ рисков, неопределённости и возможностей** - есть ли критические моменты или перспективные направления?
            4. **Основные показатели** - ключевые цифры, тренды, прогнозы.
            5. **Рыночная динамика** - влияние факторов на отрасль.
            6. Всё остальное, что может быть важно
            7. Итог.

            **Выведи отчёт (саммари) в структурированном виде:**
            """

            summarize_chain = load_summarize_chain(llm, chain_type="stuff")
            summary = summarize_chain.invoke([Document(page_content=prompt)])
            pdf['summary'] = summary['output_text']
        else:
            pdf['summary'] = ''
    if len(text_documents) == 0:
        doc['summary'] = ''
    elif len(text_documents) == 1:
        doc['summary'] = pdf['summary']
    else:
        text_document = "\n".join(text_documents)
        prompt = f"""
        **Название документа:**  {name}
        **Теги документа:** {", ".join(tags)} 
        **Краткое содержание документов с сайта (возможны html артефакты):** {additional_text}

        Перед тобой текста, полученные из PDF-документов. Там могут быть пропуски, так как изображения и таблицы не были извлечены. В этих текстах отраслевой анализ и отреслевыен отчёты.
        **Документы:**
        ```
            {text_document}
        ```

        **Твоя задача** - сделать саммари или краткое содержание этого отчёта, провести подробный и детальный отраслевой анализ на основе этого текста на русском языке. Не жалей букв, достань и саммаризируй всю полезную информацию. 
        Если информация обрывочна, попытайся восстановить логический контекст или отметь, какие данные отсутствуют.
        Данная информация потом будет использоваться для формирования стратегии компании.

        **Формат анализа:**
        1. **Ключевые выводы** - выдели главных мыслей.
        2. **Тенденции в отрасли** - связаны ли идеи с актуальными событиями или трендами? краткосрочные и долгосрочные прогнозы.
        3. **Анализ рисков, неопределённости и возможностей** - есть ли критические моменты или перспективные направления?
        4. **Основные показатели** - ключевые цифры, тренды, прогнозы.
        5. **Рыночная динамика** - влияние факторов на отрасль.
        6. Всё остальное, что может быть важно
        7. Итог.

        **Выведи отчёт (саммари)  в структурированном виде:**
        """
        summarize_chain = load_summarize_chain(llm, chain_type="stuff")
        summary = summarize_chain.invoke([Document(page_content=prompt)])
        doc['summary'] = summary['output_text']

 45%|████▌     | 96/211 [23:58<24:44, 12.91s/it]  invalid pdf header: b'RIFF\x8c'
EOF marker not found


Error processing PDF Супертренд на здоровое питание — ключевая тенденция на рынке хлебобулочных и кондитерских изделий в России: Stream has ended unexpectedly


100%|██████████| 211/211 [47:45<00:00, 13.58s/it]


### пример

In [68]:
print(docs[0]['pdfs'][0]['full_text'])

Итоги 2024 года
Оглавление
2
Вступительные слова
4
Обзор венчурного рынка
Россия
6
Обзор венчурного рынка
Средняя Азия и Закавказье
34
Обзор венчурного рынка
Восточная Европа
39
Методология
45
Российские основатели 
за рубежом
31
Команда проекта
Арсений Даббах
Основатель Dsight
Дарья Жаркова
Руководитель проекта
Владимир Хабибулин
Венчурный аналитик Dsight
Евгений Кулькин 
Дизайн и верстка
3
Вступительное слово
Дорогие друзья, читатели и коллеги по рынку!
Прошлый год стал периодом умеренного восстановления 
российского венчурного рынка. Объем инвестиций вырос на 
108%, что свидетельствует о постепенном возвращении интереса 
к венчурному капиталу.
Одним из ключевых драйверов роста стало развитие 
инвестиционных платформ и инвестиционных синдикатов, 
которые способствовали расширению доступа к капиталу для 
стартапов. В результате этого тренда мы увидели активное 
развитие инвестиций на стадии pre-IPO, а также рекордное 
количество IPO: 14 публичных размещений, из которых восемь 
были те

In [65]:
print(docs[0]['pdfs'][0]['summary'])

### 1. Ключевые выводы
- В 2024 году российский венчурный рынок показал умеренное восстановление, объем инвестиций составил $177 млн, увеличившись на 108% по сравнению с 2023 годом.
- Средний чек венчурного инвестирования вырос до $1,01 млн (+124% г/г).
- Преобладание инвестиций наблюдается в секторе B2B Software (41% от общего объема) и восстановление интереса к Consumer Goods & Services.
- Ожидаемое замедление активности сделок M&A в 2025 году связано с предыдущими покупками и высокой долговой нагрузкой.
- Рост популярности инструментов pre-IPO ввел 14 IPO в 2024 году, 8 из которых – технологические.

### 2. Тенденции в отрасли
- Краткосрочные: Увеличение числа сделок и ухода бизнес-ангелов с малочисленных сделок в условии высоких ставок, рост интереса к синдицированным сделкам.
- Долгосрочные: Ожидаемое восстановление венчурного рынка в 2025 году под воздействием нормализации геополитической ситуации и роста средней заработной платы, что повысит покупательскую способность населения.

## Добавляем теги
Теги из kksb_client_profile

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate

# Данные
data = {
    "date": "2023-10-15",
    "summary": "Саммари",
    "pdfs": [
        {
            "full_text": "Полный текст и pdf, то что получилось вытащить из content",
            "summary": "Саммари из full_text",
        },
        {
            "full_text": "Полный текст и pdf, то что получилось вытащить из content",
            "summary": "Саммари из full_text",
        }
    ]
}

llm = ChatOpenAI(model_name="gpt-4o-mini")

# Возможные теги
tags_list = [
    'Энергетика', 'Лизинг', 'Нефтегазовая промышленность', 'Пищевая промышленность',
    'Финансы', 'Органы государственного и муниципального управления', 'Материалы', 'Промышленность',
    'Телекоммуникации', 'Телекоммуникации, медиа и ИТ-технологии', 'ЖКХ', 'Услуги',
    'Электроэнергетика и ЖКХ', 'Строительные подрядчики', 'Сельское хозяйство', 'Товары первой необходимости',
    'Энергоносители', 'Товары выборочного спроса', 'Операции с недвижимым имуществом', 'Государство',
    'Недвижимость', 'Фармацевтика и здравоохранение', 'Химическая промышленность', 'Розничная торговля товарами первой необходимости',
    'Машиностроение', 'Металлургическая и горнодобывающая промышленность', 'Производство строительных материалов',
    'Розничная торговля товарами выборочного спроса', 'Финансовая деятельность', 'Легкая промышленность',
    'Транспорт и логистика', 'Производство потребительских товаров', 'Лесная, деревообрабатывающая и целлюлозно-бумажная промышленность'
]

for data in tqdm(docs):
    text_sources = [', '.join([x for x in data["tags"]if x != 'Прочее']), data["summary"], data["text"]] + [pdf["summary"] for pdf in data["pdfs"]]
    context_text = "\n".join(filter(None, text_sources))[:10_000]

    prompt = PromptTemplate(
        input_variables=["text", "tags"],
        template=(
            "Определи, какие из следующих тегов наиболее подходят для данного текста:\n\n{text}\n\n"
            "Выбери из списка: {tags}. "
            "Отвечай только списком тегов через запятую, без пояснений."
        )
    )

    response = llm.invoke(prompt.format(text=context_text, tags=", ".join(tags_list))).content.strip()
    data["sber_tags"] = [tag.strip() for tag in response.split(",") if tag.strip() in tags_list]


  0%|          | 1/211 [00:01<05:01,  1.44s/it]

['Финансы', 'Телекоммуникации', 'Фармацевтика и здравоохранение', 'Услуги', 'Пищевая промышленность', 'Производство потребительских товаров']


  1%|          | 2/211 [00:02<04:03,  1.17s/it]

['Исследование Б1', 'МСФО', 'Услуги в области устойчивого развития']


  1%|▏         | 3/211 [00:03<04:11,  1.21s/it]

...

['Дайджесты']


100%|█████████▉| 210/211 [06:46<00:03,  3.92s/it]

['Дайджесты']


100%|██████████| 211/211 [06:49<00:00,  1.94s/it]

['Дайджесты']


### Пример

In [82]:
print(data["summary"])
print('SberTags', data["sber_tags"])

# Санкционный Дайджест 2023

## 1. Ключевые выводы:
- В июле 2023 года количество новых персональных санкций против россиян составило 251, из которых 91 касалась физических лиц и 160 организаций, увеличив общее число до 14 304.
- Основные инициаторы санкций – США, Канада и Австралия. Наиболее известные фигуры среди новых подсанкционных лиц – Алексей Кудрин и ряд российских банков.
- Комитет Европарламента одобрил установление уголовной ответственности за нарушения санкционных режимов.
- Синхронизация секторальных ограничений продолжается; расширение контроля за экспортом товаров.

## 2. Тенденции в отрасли:
- Наращивание санкций и требований к соблюдению экспортных правил может привести к затруднениям в торговых операциях с Россией.
- Возможное усиление уголовных наказаний за нарушение санкций на уровне ЕС.
- Явное влияние добровольного сокращения предложения нефти на рост цен и снижение дисконта на российскую нефть.

## 3. Анализ рисков, неопределённости и возможностей:
- Увеличение р

# Сохраняемся

In [90]:
with open('docs.pkl', 'wb') as f:
    pickle.dump(docs, f)

In [101]:
from datetime import datetime

new_docs = []
for doc in docs:
    new_doc = {
        "date": doc['date'],
        "sber_tags": doc['sber_tags'],
        "summary": doc['summary'],
    }
    if isinstance(new_doc['date'], datetime):
        new_doc['date'] = new_doc['date'].strftime('%d.%m.%Y')
    pdfs = []
    for pdf in doc['pdfs']:
        pdfs.append({
            "full_text": pdf.get('full_text', ''),
            "summary": pdf['summary'],
        })
    new_doc['pdfs'] = pdfs
    new_docs.append(new_doc)

In [103]:
import json

json_lines = "\n".join(json.dumps(item) for item in new_docs)

with open("industry_docs.jsonl", "w") as f:
    f.write(json_lines)